In [86]:
import re
import os
import ast
from pprint import pprint as pprint

from datetime import date, timedelta, datetime
from nile.api.v1 import (
    clusters,
    cli,
    extractors as ne,
    aggregators as na,
    filters as nf,
    statface as ns,
    datetime as nd,
    Record,
    with_hints
)
from qb2.api.v1 import (
    filters as qf,
    extractors as qe
)

from qb2.api.v1.typing import (
    String,
    Dict,
    Float,
    UInt64,
    Optional,
    String,
    Bool,
    Union,
    List,
    Json,
    Any
)

In [87]:
os.environ['YT_TOKEN'] = os.environ.get('YT_TOKEN')
os.environ['YQL_TOKEN'] = os.environ.get('YQL_TOKEN')
os.environ['STATFACE_TOKEN'] = os.environ.get('STATFACE_TOKEN')

In [88]:
APPMETRIKA_LOG_PATH = 'logs/metrika-mobile-log/1d/'
JOB_PATH = '//home/advisor/gri201/adhoc/PHONE-5580/'

DATE_START = (date.today() - timedelta(360)).isoformat()
DATE_END = (date.today() - timedelta(1)).isoformat()
DATES_RANGE = '{' + DATE_START + '..' + DATE_END + '}'

In [89]:
cluster = clusters.Hahn()

job = cluster.job().env(
    yt_spec_defaults=dict(
        pool_trees=["physical"],
        tentative_pool_trees=["cloud"]
    )
)

In [90]:
MANUFACTURERS_LIST = {
    'Huawei',
    'Samsung'
}

MODELS_LIST = {
    'MAR-LX1M',
    'SM-A530F',
    'COR-L29',
    'SNE-LX1',
    'SM-A505F',
    'HRY-LX1T',
    'SM-A750F'
}

In [91]:
def get_mac(a, b):
    try:
        return str(dict(zip(json.loads(a), json.loads(b))).get('wlan0'))
    except ValueError:
        return None

@with_hints(output_schema=dict(
    device_id=str, 
    app_id=Optional[String], 
    date=Optional[String], 
    imei=Optional[String], 
    gaid=Optional[String],
    mac=Optional[String],
    manufacturer=Optional[String], 
    model=Optional[String]
))
def unfold_imei_mapper(records):
    for rec in records:
        if rec['imei'] is None:
            yield Record(
                device_id = rec['device_id'], 
                app_id = rec['app_id'],
                gaid = rec['gaid'],
                date = rec['date'],
                imei = rec['imei'],
                manufacturer = rec['manufacturer'],
                mac = get_mac(rec['interfaces_names'], rec['interfaces_macs']),
                model = rec['model']
            )
        else:
            imei_list = json.loads(rec['imei'])
            for imei in imei_list:
                yield Record(
                    device_id = rec['device_id'], 
                    app_id = rec['app_id'],
                    gaid = rec['gaid'],
                    date = rec['date'],
                    imei = imei,
                    manufacturer = rec['manufacturer'],
                    mac = get_mac(rec['interfaces_names'], rec['interfaces_macs']),
                    model = rec['model']
                )

In [92]:
job = cluster.job()

log = job.table(APPMETRIKA_LOG_PATH + DATES_RANGE)
prepared = log.qb2(
    log='metrika-mobile-log',
    fields=[
        'device_id',
        'date',
        qe.log_field('AppID', default="unknown").rename('app_id'),
        qe.log_field('ADVID', default="unknown").rename('gaid'),
        qe.log_field('NetworksInterfaces_Names', default="unknown").rename('interfaces_names'),
        qe.log_field('NetworksInterfaces_Macs', default="unknown").rename('interfaces_macs'),
        qe.log_field('Imei', default=None).rename('imei'),
        qe.log_field('Manufacturer', default="unknown").rename('manufacturer'),
        qe.log_field('Model', default="unknown").rename('model'),
    ],
    filters=[
        qf.equals('api_key_str', '13'),
        qf.equals('app_id', 'ru.sberbankmobile'),
        qf.one_of('manufacturer', MANUFACTURERS_LIST),
        qf.one_of('model', MODELS_LIST),
        qf.defined('imei', 'gaid', 'interfaces_names')
    ]
).map(
    unfold_imei_mapper,
    intensity='cpu'
).filter(
    qf.defined('imei', 'mac')
).groupby(
    'device_id', 'gaid', 'mac', 'date'
).top(
    1,
    by='imei'
).project(
    ne.all()
).put(
    JOB_PATH + 'did_imei_gaid_mac'
)

job.run()

VBox()